# 패키지 선언

In [2]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import requests
import re
import pandas as pd
import numpy as np
import json
from random import *
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--no-referrers')
options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36')

executable_path = '/opt/google/chrome/chromedriver' # chrome driver의 경로
driver = webdriver.Chrome(executable_path=executable_path, options=options)

# isbn 데이터 불러오기 - 알라딘 베스트셀러 2020

In [3]:
best_2020 = pd.read_csv('./book_data/nationallib/bestseller_aladin/NL_BO_BEST_BOOK_HISTORY_ARCHIVE_202012.csv', low_memory=False)
best_isbns = best_2020.loc[:, 'isbn13'].tolist()

len(best_isbns)

190852

# 알라딘 크롤링

In [66]:
isbn = best_isbns[50]
isbn

'9788971996676'

In [67]:
book_spec = 'https://www.aladin.co.kr/shop/wproduct.aspx?isbn='
url = book_spec + isbn
driver.implicitly_wait(2)
driver.get(url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

In [77]:
page_source

'<html xmlns="http://www.w3.org/1999/xhtml"><head>\n    <title id="Title">알라딘: 담론</title>\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge">\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <link rel="alternate" media="only screen and (max-width: 640px)" href="https://www.aladin.co.kr/m/mproduct.aspx?ItemId=56032296"><meta property="og:type" content="books.book">\n<meta property="books:isbn" content="9788971996676">\n<meta property="og:image" content="https://image.aladin.co.kr/product/5603/22/cover500/8971996676_3.jpg">\n<meta property="og:site_name" content="aladin">\n<meta name="description" content="<강의> 출간 이후 10년 만에 출간되는 선생의 ‘강의록’이다. 이 책은 동양고전 말고도 <나무야 나무야>, <감옥으로부터의 사색> 등 선생의 다른 책에 실린 글들을 교재 삼아 평소에 이야기하신 ...">\n<meta itemprop="datePublished" content="2015-04-20">\n<meta property="og:description" content="<강의> 출간 이후 10년 만에 출간되는 선생의 ‘강의록’이다. 이 책은 동양고전 말고도 <나무야 나무야>, <감옥으로부터의 사색> 등 선생의 다른 책에 실린 글들을 교재 삼아 평소에 이야기하신 ...">\n<meta name="title" conten

## 제목

In [30]:
title = soup.find("a", {'class' : 'Ere_bo_title'}).text
try:
    sub_title = soup.find("span", {'class' : 'Ere_sub1_title'}).text
except:
    sub_title = '없음'

print(title + ' ' + sub_title)

담론 - 신영복의 마지막 강의


## 저자

In [31]:
author_li = soup.find("li", {'class' : 'Ere_sub2_title'}).text
p = re.compile('.*\\xa0\(지은이\)')
author_text = p.match(author_li).group()
author = author_text.replace('\xa0(지은이)', '')

print(author)

신영복


## 옮긴이

In [32]:
try:
    translator_text = re.search('\(지은이\).*\(옮긴이\)', author_li)
    translator = translator_text.group().replace('(지은이),', '').replace('\xa0(옮긴이)', '')
except:
    translator = '없음'
    
print(translator)

없음


## 카테고리

In [34]:
categories = soup.find("ul", {'id' : 'ulCategory'})
category = categories.find_all("li")

category_arr = [x.text.split('\xa0') for x in category]
each_category = [' '.join(x).replace(' 접기', '') for x in category_arr]

for i in each_category:
    print(i)

국내도서 > 인문학 > 교양 인문학
국내도서 > 추천도서 > 알라딘 독자 선정 올해의 책 > 2015년 > 인문.교양 분야 베스트 5
국내도서 > 추천도서 > 외부/전문기관 추천도서 > 현대경제연구원이 추천한 휴가철.. > 2015년


## 편집장의 선택

In [35]:
try:
    editor_comment_titlel = soup.find('div', {'class' : 'Ere_fs18 Ere_sub_gray3 Ere_str'})
    editor_comment_title = editor_comment_titlel.text
    editor_comment_contents = editor_comment_titlel.next_sibling.next_siblings
    full_contents = [x for x in editor_comment_contents]
    full_content = '\n'.join(list(filter(lambda x: len(x) > 50, full_contents)))
except:
    full_content = '없음'

print(full_content)

공적 자리에서 사람을 지칭할 때는 직함을 부르거나 씨를 붙이거나 이름만 부르는 게 예의지만, 그럼에도 자연스레 이름 뒤에 선생, 선생님을 붙이게 되는 경우가 있다. 바로 신영복 선생이 그렇다. 그가 오랜 기간 교수로 지내서, 사회의 원로라서 그리 된 게 아니라, 그의 삶과 사상을 담아내는 그릇이 ‘강의’이고, 그를 아는 많은 이가 ‘강의’로 그를 만났기 때문이다. 그렇기에 마지막 강의를 마치고 그간의 강의를 한데 모아 정리한 <담론>은 신영복 삶과 사상의 정수라 해도 부족함이 없겠다.

마지막 강의는 세계를 해석하는 창으로서 ‘고전 읽기 교실’과 이를 바탕으로 자기 삶을 돌아보고 인간을 이해하는 과정으로서 ‘인간학 교실’로 이루어진다. 세계와 인간, 세계와 나가 그렇듯 강의를 꿰뚫는 핵심은 관계다. 신영복은 모든 담론의 중심에 관계를 두고 이야기를 풀어가며, 관계가 확장되는 변방에서 변화와 창조의 가능성을 살핀다. 자신의 강의 역시 같은 맥락으로 읽히길 바라며, 이 책을 우연한 점으로 삼아 각자가 선과 면을 만들어 가길 권한다. 물론 쉬운 일은 아니겠지만, 그는 훌륭한 스승답게 방향뿐 아니라 도전에 필요한 용기까지 강의에 담아 두었다. 책으로 만난 그의 제자로서, 고마우면서도 아쉬운 마지막 강의를 곱씹는다. 


## isbn10

In [39]:
try:
    meta_image = soup.find('meta', {'property' : 'og:image'})
    meta_arr = meta_image.attrs['content'].split('/')
    isbn_10 = re.sub('_\d.jpg', '', meta_arr[-1])
except:
    isbn_10 = '없음'
    
print(meta_image)
print(meta_arr)
print(isbn_10)

<meta content="https://image.aladin.co.kr/product/5603/22/cover500/8971996676_3.jpg" property="og:image"/>
['https:', '', 'image.aladin.co.kr', 'product', '5603', '22', 'cover500', '8971996676_3.jpg']
8971996676


## 책 소개

## 목차

## 책 속에서

## 저자 소개

## 출판사 제공 책 소개

# 교보 크롤링 코드

In [ ]:
def parse_kyobo(isbn):
    try:
        book_spec = 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode='
        url = book_spec + isbn
        driver.get(url)
        page_source = driver.page_source

        soup = BeautifulSoup(page_source, 'html.parser')

        # title, author
        title_soup = soup.select_one('meta[name="title"]')
        author_soup = soup.select_one('meta[name="author"]')
        title = title_soup['content'][:-7]
        author = author_soup['content']
        book_obj = {'isbn' : isbn, 'title' : title, 'author' : author, 'total_ex' : '', 
                    'book_som_comment' : '', 'table_contents' : '', 'book_inside' : '', 'publisher_review' : ''}

        # total_ex
        try:
            ex1 = soup.select_one('.title_detail_basic2').get_text().strip()
            ex2 = soup.select_one('.box_detail_article').get_text().strip()
            try:
                ex3_a = soup.select_one('.box_detail_comment').get_text().strip()
                ex3 = re.sub('[\\n|\\t]{2,}', ' ', ex3_a)
                total_ex = ex1 + '\n\n'+ ex2 + '\n\n' + ex3
            except:
                total_ex = ex1 + '\n\n'+ ex2
        except:
            total_ex = ''
        book_obj['total_ex'] = total_ex

        spec_tags = soup.select('.box_detail_article')

        # book_intro
        book_intro = spec_tags[0].get_text().strip()
        info_collection = ['목차', '책 속으로', '출판사 서평', '북소믈리에 한마디!']

        table_contents = ''
        book_inside = ''
        publisher_review = ''
        book_som_comment = ''

        for tag in spec_tags:

            # table_contents
            if str(tag.previous_element.previous_element) == info_collection[0]:  # 목차
                try:
                    table_contents = tag.get_text().strip().replace('\n\n', '\n')
                except:
                    table_contents = ''
            book_obj['table_contents'] = table_contents

            # book_inside
            if str(tag.previous_element.previous_element) == info_collection[1]:  # 책 속으로
                try:
                    inside_book = tag.get_text().strip().replace('\n\n', '\n').replace('\t', '').replace('\n닫기', '')
                    useless_start = inside_book.find('더보기')
                    book_inside = inside_book[useless_start+5:]
                except:
                    book_inside = ''
            book_obj['book_inside'] = book_inside

            # publisher_review
            if str(tag.previous_element.previous_element) == info_collection[2]:  # 출판사 서평
                try:
                    publisher_reviews = tag.get_text().replace('\n\n', '\n').replace('\t', '').replace('\n닫기', '')
                    useless_start = publisher_reviews.find('더보기')
                    publisher_review = publisher_reviews[useless_start+5:]
                except:
                    publisher_review = ''
            book_obj['publisher_review'] = publisher_review

            # book_sommelier_comment
            if str(tag.previous_element.previous_element) == info_collection[3]:  # 북소믈리에 한마디!
                try:
                    book_som_comment = tag.get_text().strip()
                except:
                    book_som_comment = ''
            book_obj['book_som_comment'] = book_som_comment
    except UnexpectedAlertPresentException as e:
        print(e.__dict__["msg"])
        
    return book_obj